# MSCF 46982 Market Microstructure and Algorithmic Trading
# Fall 2018 Mini 2

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", specify your name and that of your collaborator, and remove the `notimplemented` exception.



---

In [1]:
NAME: "Ze Yang"
COLLABORATOR: ""

## Probability of Informed Trading (PIN)

The stock market is filled with informed and uninformed traders (participants).  You may assume that a highly efficient marketplace results in the probability of trading with an informed participant is low.  But how do we compute this probability?  The PIN statistic developed by Easley, O'Hara et al provides a value (between 0 and 1) of the probability of informed trading.  In this assignment, you are asked to calculate the PIN statistic for a single stock.  The steps will include computing the number of buy and sell ordrs using the Lee and Ready algorithm, implementing the PIN ratio and the log likelihood and finally using the the `scipy optimize` package to fit the PIN parameters and calculate the PIN ratio.

Before beginning this assignment, make sure the `scipy` package is installed by running the following from the anaconda prompt
```
conda install -c anaconda scipy
```

We start by opening a connection to the NYSE Daily TAQ database. The generated integer saved in `h` is a Kdb+ file handle.  It will be used for all database communication.

NOTE: The database is located on a CMU server - behind the firewall.  If you are doing this assignment from home, you will need to connect to the CMU network using the Cisco AnyConnect VPN software.

In [2]:
\c 5 100
h:0N!hopen `$":tpr-mscf-kx.tepper.cmu.edu:5000:mscf2018:mmat46982"

6i


### Part A (1 points)

Complete the `landr` function so that it downloads the total number of buy and sell trades for a given list of symbols on a given date and condition codes

The returned table should have 4 columns: `date,sym,B,S` where the `B` column indicates the total number of buy trades and the `S` column indicates the total number of sell trades.

In [3]:
/ (s)ym, trade (c)ondition codes, (d)a(t)e
landr:{[s;c;dt]
 / YOUR CODE HERE
 t:select date,sym,time,price from trade where date=dt,sym in s,cond in c;
 t:aj[`sym`time;t]select sym,time,bid,ask from nbbo where date=dt;
 t:update mid:"e"$.5*ask+bid,tick:fills -1 0N 1@1+signum price-prev price by sym from t;
 t:update side:?[price>mid;1;?[price<mid;-1;tick]] by sym from t;
 t:0!select B:sum side>0, S:sum side<0 by date,sym from t;
 t}

In [4]:
/ pass function to the database for execution
h (landr;`BAC`MSFT;" ";2018.09.04)

date       sym  B     S    
---------------------------
2018.09.04 BAC  34432 34422
2018.09.04 MSFT 26244 28019


Your results should match the following:
```
date       sym  B     S    
---------------------------
2018.09.04 BAC  34432 34422
2018.09.04 MSFT 26244 28019
```

In [5]:
rnd:{x*"j"$y%x}
assert:{if[not x~y;'`$"expecting '",(-3!x),"' but found '",(-3!y),"'"]}
/ confirm all columns are included
assert[`date`sym`B`S] cols h (landr;`BAC;" ";2018.09.04)
/ confirm schema is correct
assert["dsii"] first flip value meta h (landr;`BAC;" ";2018.09.04)
/ confirm query only selects specified dates
assert[2018.09.04+0 1]  exec distinct date from raze {h (landr;`BAC;" ";x)} each 2018.09.04 + 0 1
/ confirm only requested syms have been returned
assert[`BAC`TSLA] exec distinct sym from h (landr;`BAC`TSLA;" ";2018.09.04)
/ confirm correct number of rows have been returned
assert[2] count h (landr;`BAC`TSLA;" ";2018.09.04)
/ confirm only selected condition codes have been returned
assert[1i] first (h (landr;`BAC;"O";2018.09.04))`B
assert[34432i] first (h (landr;`BAC;" ";2018.09.04))`B

### Part B (1 points)

In class, we implemented a version of the PIN ratio which differentiated between the rate of buy and sell uninformed participants. Implement a simplified PIN calculation, which does not differentiate between the two.

$$\text{PIN} =\frac{\alpha\mu}{2\epsilon+\alpha\mu}$$


Complete the `pin` function so that it computes the PIN ratio given the simplified parameters "emad" $\epsilon$, $\mu$, $\alpha$, $\delta$


In [6]:
/ (e)psilon, (m)u, (a)lpha, (d)elta
pin:{[emad]
 / YOUR CODE HERE
 p:(emad[2]*emad[1])%((2f*emad[0])+emad[2]*emad[1]);
 p}

In [7]:
/ test pin function
pin 10000 100 .5 .5

0.002493766


Your results should match the following:
```
0.002493766
```

In [8]:
/ confirm PIN limits
assert[1f] pin 0 1 .5 .5
assert[0f] pin 1 0 .5 .5
assert[0f] pin 1 1 0 .5
assert[.5] pin 2 2 2 .5

### Part C (1 points)

This assignment requires us to implement the PIN log likelihood in python.  Let's get some practice by implementing the PIN function in python.

Complete the python `pin` function so that it computes the PIN ratio given the simplified parameters "emad" $\epsilon$, $\mu$, $\alpha$, $\delta$


In [9]:
/%python
# (e)psilon, (m)u, (a)lpha, (d)elta
def pin(emad):
 return emad[2]*emad[1]/(2*emad[0]+emad[2]*emad[1])

"p)# (e)psilon, (m)u, (a)lpha, (d)elta"
"p)def pin(emad):\n return emad[2]*emad[1]/(2*emad[0]+emad[2]*emad[1])"


In [10]:
/%python
# test pin function
print(pin([10000,100,.5,.5]))

"p)# test pin function"
"p)print(pin([10000,100,.5,.5]))"
0.0024937655860349127


Your results should match the following:
```
0.0024937655860349127
```

In [11]:
/%python
# confirm PIN limits
print("test 1")
assert 1==pin([0,1,.5,.5])
print("test 2")
assert 0==pin([1,0,.5,.5])
print("test 3")
assert 0==pin([1,1,0,.5])
print("test 4")
assert .5==pin([2,2,2,.5])

"p)# confirm PIN limits"
"p)print(\"test 1\")"
"p)assert 1==pin([0,1,.5,.5])"
"p)print(\"test 2\")"
"p)assert 0==pin([1,0,.5,.5])"
"p)print(\"test 3\")"
"p)assert 0==pin([1,1,0,.5])"
"p)print(\"test 4\")"
"p)assert .5==pin([2,2,2,.5])"
test 1
test 2
test 3
test 4


### Part D (1 points)

In class, we implemented a version of the PIN ratio which differentiated between the rate of buy and sell uninformed participants. Implement a simplified PIN calculation which does not differentiate between the two.

$$\log \mathcal{L} = \sum_{d=1}^n -2\epsilon + M_d \log x + (B_d+S_d)\log(\mu + \epsilon) $$
$$ + \sum_{d=1}^n \log \left( (1-\alpha)x^{S_d+B_d-M_d} + \alpha(1-\delta)\exp(-\mu)x^{S_d-M_d} + \alpha\delta\exp(-\mu)x^{B_d-M_d}\right)$$

where:

$$M_d = \min(B_d,S_d) + \frac{\max(B_d,S_d)}{2}$$
$$x = \frac{\epsilon}{\epsilon + \mu} $$

Complete the `pin_ll` function so that it computes the log likelihood given the simplified parameters "emad" $\epsilon$, $\mu$, $\alpha$, $\delta$ and `B`,`S`

For numeric stability, your function should enforce the following limits:

1. $\epsilon$ and $\mu$ are rates, so prevent the values from being negative.  Floor the passed in values to 0.
2. $\alpha$ and $\delta$ are probabilities, so limit the values to be between 0 and 1. Floor the passed in values to 0 and cap them to 1.
3. Prevent the function from taking the log of 0 by limiting the values passed to the $\log$ operator in the second summation to 1e-300.  Floor the values passed to $\log$ to 1e-300.



In [12]:
/ (e)psilon, (m)u, (a)lpha, (d)elta, (B)uy count, (S)ell count
pin_ll:{[emad;B;S]
 / YOUR CODE HERE
 e:emad[0];m:emad[1];a:emad[2];d:emad[3];
 e:0|e;m:0|m;a:1&0|a;d:1&0|d;
 M:(B&S)+.5*B|S;
 x:e%(m+e);
 ll:sum neg[e+e]+((B+S)*log(e+m))+M*log[x];
 ll+:sum log 1e-300|((1-a)*(x xexp S+B-M))+((a*1-d)*(exp[neg m]*x xexp S-M))+(a*d*exp[neg m]*x xexp B-M);
 ll}

In [13]:
t:raze {h (landr;`LOB;" ";x)} each 2018.09.01 + til 30
.p.set[`B] get `B set t`B
.p.set[`S] get `S set t`S

In [14]:
/ test pin function
pin_ll[10000 1000 .5 .5;B;S]

-318672.3


Your results should match the following:
```
-318672.3
```

In [15]:
/ confirm pin_ll calculation
assert[-3723337] rnd[1] pin_ll[100000 100000 .5 2;B;S]
assert[-3735122] rnd[1] pin_ll[100000 100000 2 .5;B;S]
assert[-3723337] rnd[1] pin_ll[100000 100000 .5 .5;B;S]
assert[-318672] rnd[1] pin_ll[10000 10000 .5 .5;B;S]
assert[7992] rnd[1] pin_ll[1000 1000 .5 .1;B;S]


### Part E (1 points)

We will need to optimize this function, so convert the `pin_ll` to python.

Complete the python `pin_ll` function so that it computes the log likelihood given the simplified parameters "emad" $\epsilon$, $\mu$, $\alpha$, $\delta$ and `B`,`S`



In [16]:
/%python
import numpy as np
def pin_ll(emad,B,S):
 # YOUR CODE HERE
 e, m, a, d = emad;
 e, m, a, d = max(0, e), max(0, m), np.clip(a, 0, 1), np.clip(d, 0, 1)
 M = np.fmin(B, S) + np.fmax(B, S)/2
 x = e/(m+e)
 ll = -2*e*len(M) + M.sum()*np.log(x) + np.log(m+e)*(B+S).sum()
 ll += np.log(np.fmax(1e-300, (1-a)*x**(S+B-M) + 
  a*(1-d)*np.exp(-m)*x**(S-M) + a*d*np.exp(-m)*x**(B-M) )).sum()
 return ll

"p)import numpy as np"
"p)def pin_ll(emad,B,S):\n # YOUR CODE HERE\n e, m, a, d = emad;\n e, m, a, d = max(0, e), max(0,..


In [17]:
/%python
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
# test function
print(pin_ll([10000,100,.5,.5],B,S))

"p)import math"
"p)import numpy as np"
"p)import pandas as pd"
"p)import matplotlib.pyplot as plt"
"p)from scipy.optimize import minimize"
"p)# test function"
"p)print(pin_ll([10000,100,.5,.5],B,S))"
-318672.302919


Your results should match the following:
```
-318672.3029190693
```

In [18]:
/%python
# confirm pin_ll calculation
print("test 1")
assert -3723337 == round(pin_ll([100000,100000,.5,.5],B,S),0)
print("test 2")
assert -3723337 == round(pin_ll([100000,100000,.5,2],B,S),0)
print("test 3")
assert -3735122 == round(pin_ll([100000,100000,2,.5],B,S),0)
print("test 4")
assert -318672 == round(pin_ll([10000,10000,.5,.5],B,S),0)
print("test 5")
assert 7992 == round(pin_ll([1000,1000,.5,.1],B,S),0)


"p)# confirm pin_ll calculation"
"p)print(\"test 1\")"
"p)assert -3723337 == round(pin_ll([100000,100000,.5,.5],B,S),0)"
"p)print(\"test 2\")"
"p)assert -3723337 == round(pin_ll([100000,100000,.5,2],B,S),0)"
"p)print(\"test 3\")"
"p)assert -3735122 == round(pin_ll([100000,100000,2,.5],B,S),0)"
"p)print(\"test 4\")"
"p)assert -318672 == round(pin_ll([10000,10000,.5,.5],B,S),0)"
"p)print(\"test 5\")"
"p)assert 7992 == round(pin_ll([1000,1000,.5,.1],B,S),0)"
test 1
test 2
test 3
test 4
test 5


### Part F (1 points)

To find the values of $\epsilon$, $\mu$, $\alpha$, and $\delta$ that maximize the log likelihood we can use the `minimize` function in the python `scipy optimize` package.  Since this is a minimization algorithm, we must first implement our `objective` function that gets smaller as the log likelihood gets bigger.

Complete the python `objective` function so that it returns the negative of the log likelihood.



In [19]:
/%python
def objective(emad,B,S):
 # YOUR CODE HERE
 return -pin_ll(emad,B,S)

"p)def objective(emad,B,S):\n # YOUR CODE HERE\n return -pin_ll(emad,B,S)"


In [20]:
/%python
# test function
print(objective([10000,100,.5,.5],B,S))

"p)# test function"
"p)print(objective([10000,100,.5,.5],B,S))"
318672.302919


Your results should match the following:
```
318672.3029190693
```

In [21]:
/%python
# confirm pin_ll calculation
print("test 1")
assert 318672 == round(objective([10000,100,.5,.5],B,S),0)
print("test 2")
assert 3723337 == round(objective([100000,100000,.5,2],B,S),0)

"p)# confirm pin_ll calculation"
"p)print(\"test 1\")"
"p)assert 318672 == round(objective([10000,100,.5,.5],B,S),0)"
"p)print(\"test 2\")"
"p)assert 3723337 == round(objective([100000,100000,.5,2],B,S),0)"
test 1
test 2


### Part G (1 points)

Even though we enforced that the values of $\epsilon$, and $\mu$ never go negative and we bounded $\alpha$ and $\delta$ to be between 0 and 1, we should also create a set of bounds for the `minimize` function so that it does not try to generate an illogical solution.

Create a variable called `bounds` which has list of bounds - one for each of parameters expected by the `objective` function.  The order of the bounds must match the order of the coefficients. You may refer to the `minimize` [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) for specifics on the expected bounds structure.

In [22]:
/%python
bounds = [(0,None),(0,None),(0,1),(0,1)]
# YOUR CODE HERE



"p)bounds = [(0,None),(0,None),(0,1),(0,1)]"
"p)# YOUR CODE HERE"
"p)"


In [23]:
/%python
# display bounds
print(bounds)

"p)# display bounds"
"p)print(bounds)"
[(0, None), (0, None), (0, 1), (0, 1)]


In [24]:
/%python
# confirm pin_ll calculation
assert 0 == bounds[0][0]
assert None == bounds[0][1]
assert 0 == bounds[1][0]
assert None == bounds[1][1]
assert 0 == bounds[2][0]
assert 1 == bounds[2][1]
assert 0 == bounds[3][0]
assert 1 == bounds[3][1]



"p)# confirm pin_ll calculation"
"p)assert 0 == bounds[0][0]"
"p)assert None == bounds[0][1]"
"p)assert 0 == bounds[1][0]"
"p)assert None == bounds[1][1]"
"p)assert 0 == bounds[2][0]"
"p)assert 1 == bounds[2][1]"
"p)assert 0 == bounds[3][0]"
"p)assert 1 == bounds[3][1]"
"p)"


### Part H (1 points)

The last step is to perform the minimization on the objective function.

Create a variable called `sol` which stores the result of the `minimize` function.  You should specify the following arguments:

1. The objective function
2. The vector of initial guesses
3. The `args` option which holds extra arguments that need to passed to the objective function (the buy and sell vector)
4. A `method` option which specifies minimization method that handles non-linear functions
5. The `bounds` options which holds the bounds vector

If you desire to see the intermediate parameter vectors, you may implement a function to be passed to the `callback` option which prints the passed in argument.

NOTE: It is important to supply a reasonable guess of the initial parameters.  You may need to try multiple times - and think what the values actually mean to get `minimize` to find a valid solution.

In [25]:
/%python
sol = None
# YOUR CODE HERE
from scipy.optimize import minimize
sol = minimize(objective, x0=np.array([100, 100, .5, .5]), args=(B,S), bounds=bounds)


"p)sol = None"
"p)# YOUR CODE HERE"
"p)from scipy.optimize import minimize"
"p)sol = minimize(objective, x0=np.array([100, 100, .5, .5]), args=(B,S), bounds=bounds)"


In [26]:
/%python
# test function
print(sol)

"p)# test function"
"p)print(sol)"
      fun: -27985.395704062761
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0007276 ,  0.0003638 , -0.00545697,  0.00327418])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 130
      nit: 20
   status: 0
  success: True
        x: array([ 146.74390907,  119.87847046,    0.475773  ,    0.53445595])


Your results should be similar to the following (specifically, `Success: True`):
```
     fun: -27985.395704447892
     jac: array([ 0.00024414, -0.00048828,  0.00244141, -0.00097656])
 message: 'Optimization terminated successfully.'
    nfev: 124
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([146.74509945, 119.87358384,   0.4758344 ,   0.53436732])
```

In [27]:
/%python
# confirm successful optimization
print("test 1")
assert sol.success

"p)# confirm successful optimization"
"p)print(\"test 1\")"
"p)assert sol.success"
test 1


### Part I (0 points)

We can now determine the probability of informed trading (PIN).

In [28]:
/%python
print(pin(sol.x))

"p)print(pin(sol.x))"
0.16271394156


## However...
The results above seems to be a local optimum which is very sensitive to starting point x0.

In [29]:
/%python
sol2 = minimize(objective, x0=np.array([10, 10, 0.1, 0.1]), args=(B,S), bounds=bounds)
print(sol2)
print(pin(sol2.x))

sol3 = minimize(objective, x0=np.array([10000, 100, 0.1, 0.1]), args=(B,S), bounds=bounds)
print(sol3)
print(pin(sol3.x))

"p)sol2 = minimize(objective, x0=np.array([10, 10, 0.1, 0.1]), args=(B,S), bounds=bounds)"
"p)print(sol2)"
"p)print(pin(sol2.x))"
"p)"
"p)sol3 = minimize(objective, x0=np.array([10000, 100, 0.1, 0.1]), args=(B,S), bounds=bounds)"
"p)print(sol3)"
"p)print(pin(sol3.x))"
      fun: -27747.482101081034
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.        ,  0.51295501,  0.        ,  0.        ])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 70
      nit: 12
   status: 0
  success: True
        x: array([ 175.26476666,    0.        ,    1.        ,    1.        ])
0.0


__main__:7: RuntimeWarning: divide by zero encountered in log
__main__:9: RuntimeWarning: divide by zero encountered in power
__main__:9: RuntimeWarning: invalid value encountered in multiply
__main__:9: RuntimeWarning: invalid value encountered in fmax
__main__:9: RuntimeWarning: overflow encountered in power
/Users/zed/anaconda/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
__main__:9: RuntimeWarning: invalid value encountered in double_scalars


      fun: 317265.12010120775
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 37.33439371,   0.        ,  19.00480129,   0.        ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 35
      nit: 2
   status: 0
  success: True
        x: array([  9.96265979e+03,   1.00000000e+02,   0.00000000e+00,
         1.00000000e-01])
0.0
